<a href="https://colab.research.google.com/github/dharsanakb/FML/blob/main/FML_AdaBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

data = {
    "Gender": ["M", "F", "M", "F", "M"],
    "Age": [38, 52, 45, 29, 61],
    "Income": [420000, 360000, 780000, 300000, 500000],
    "Smoking": [0, 5, 0, 12, 8],
    "y": [-1, +1, +1, +1, +1]  # Illness: No=-1, Yes=+1 (with noise)
}

df = pd.DataFrame(data)

n = len(df)
weights = np.ones(n) / n   # initial weights = 0.2

def stump_smoking(x):
    return 1 if x >= 1 else -1

def stump_age(x):
    return 1 if x >= 45 else -1

alphas = []
stumps = []

def train_round(stump_func, feature, weights):
    predictions = df[feature].apply(stump_func).values
    incorrect = predictions != df["y"].values
    error = np.sum(weights * incorrect)

    alpha = 0.5 * np.log((1 - error) / error)

    # Update weights
    weights = weights * np.exp(-alpha * df["y"].values * predictions)
    weights = weights / np.sum(weights)

    return alpha, predictions, weights, error

# -------- ROUND 1 (Smoking) --------
alpha1, pred1, weights, err1 = train_round(stump_smoking, "Smoking", weights)
alphas.append(alpha1)
stumps.append(("Smoking", stump_smoking))

# -------- ROUND 2 (Age) --------
alpha2, pred2, weights, err2 = train_round(stump_age, "Age", weights)
alphas.append(alpha2)
stumps.append(("Age", stump_age))

# -------- ROUND 3 (Smoking again) --------
alpha3, pred3, weights, err3 = train_round(stump_smoking, "Smoking", weights)
alphas.append(alpha3)
stumps.append(("Smoking", stump_smoking))

def strong_classifier(row):
    score = 0
    for alpha, (feature, stump) in zip(alphas, stumps):
        score += alpha * stump(row[feature])
    return np.sign(score)

df["Final_Prediction"] = df.apply(strong_classifier, axis=1)

print("Alphas:", alphas)
print("\nFinal Predictions:")
print(df[["Age", "Smoking", "y", "Final_Prediction"]])

Alphas: [np.float64(0.6931471805599453), np.float64(0.9729550745276566), np.float64(0.45814536593707733)]

Final Predictions:
   Age  Smoking  y  Final_Prediction
0   38        0 -1              -1.0
1   52        5  1               1.0
2   45        0  1              -1.0
3   29       12  1               1.0
4   61        8  1               1.0
